## Segmenting and Clustering Neighborhoods in Toronto 

In [3]:
pip install bs4

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

Read URL

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

Parse HTML link and look at the elements(tags)

In [6]:
soup = BeautifulSoup(page, 'html.parser')
#print(soup.prettify())

Find the postal table

In [7]:
all_tables=soup.find_all("table")

In [8]:
table=soup.find('table', class_='wikitable')

Create columns and find the data, then append data to those columns

In [9]:
A=[]
B=[]
C=[]
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Rename columns and replace "\n" with blank

In [10]:
df=pd.DataFrame(A,columns=['Postal code'])
df['Borough']=B
df['Neighborhood']=C
df['Postal code'] = df['Postal code'].replace('\n','', regex=True)
df['Borough'] = df['Borough'].replace('\n','', regex=True)
df['Neighborhood'] = df['Neighborhood'].replace('\n','', regex=True)
df['Neighborhood'] = df['Neighborhood'].replace('/',',', regex=True)

Only process the cells that have an assigned borough

In [11]:
df2 = df[df['Borough'] != 'Not assigned']
df2

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


Print the number of rows of df2

In [12]:
df2.shape

(103, 3)

In [13]:
pip install geocoder

^C
Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pip/__main__.py", line 16, in <module>
    from pip._internal import main as _main  # isort:skip # noqa
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pip/_internal/__init__.py", line 40, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pip/_internal/cli/autocompletion.py", line 8, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdopt

Import the csv file and change a column name

In [15]:
df= pd.read_csv("http://cocl.us/Geospatial_data")
df.rename(columns = {'Postal Code':'Postal_Code'},inplace = True) 

Join two dataframes

In [16]:
Geocode=df2.merge(df,left_on='Postal code',right_on='Postal_Code')
del Geocode['Postal_Code']
Geocode

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509


In [18]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [52]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import requests 
from pandas.io.json import json_normalize 
import numpy as np

In [21]:
address= 'Toronto, Ontario'
geolocator = Nominatim(user_agent='toronto_explorer')
Location = geolocator.geocode(address)
Latitude = Location.latitude
Longitude= Location.longitude 
print('The coordinates of toronto are {},{}'.format(Latitude,Longitude))

The coordinates of toronto are 43.6534817,-79.3839347


In [26]:
Toronto_data = Geocode[Geocode['Borough'].str.contains ('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [31]:
map_toronto = folium.Map(location=[Latitude,Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format('Neighborhood', 'Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [35]:
CLIENT_ID = 'V3LUEYKPVS5AMGAYSA3RPKA3TGDDF10XMGNWCU55D1XRJMO5' 
CLIENT_SECRET = 'PD2NMQTYKCWDX5OFW41YI2FPWHGJXQYVP3BA0GYF0IQKHEMH' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

In [40]:
LIMIT= 100
def getNearbyVenues(names, latitudes, longitudes, radius=500) :

  venues_list = []
  for name,lat,lng in zip(names,latitudes,longitudes) :
    #print(name)
    url= 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                              CLIENT_SECRET,
                                                                                                                              VERSION,
                                                                                                                              lat,
                                                                                                                              lng,
                                                                                                                              radius,
                                                                                                                              LIMIT)
    
    results= requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(name,
                         lat,
                         lng,
                         v['venue']['name'],
                         v['venue']['location']['lat'],
                         v['venue']['location']['lng'],
                         v['venue']['categories'][0]['name']) for v in results])
    nearby_venues= pd.DataFrame([Item for venue_list in venues_list for Item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

                  
  return(nearby_venues)

In [41]:
toronto_venues= getNearbyVenues(Toronto_data['Neighborhood'],
                                Toronto_data['Latitude'],
                                Toronto_data['Longitude'])

In [42]:
toronto_venues.shape

(1612, 7)

In [46]:
toronto_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [48]:
toronto_hot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed = [toronto_hot.columns[-1]] + list(toronto_hot.columns[:-1])
toronto_hot = toronto_hot[fixed]
fixed2= [toronto_hot.columns[-1]] + fixed

In [97]:
toronto_grouped = toronto_hot.groupby('Neighborhood').mean().reset_index()

In [50]:
def return_most_common_venues(row, num_top_venues) :
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)

  return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10
indicators= ['st', 'nd', 'rd']
columns= ['Neighborhood']

for ind in np.arange(num_top_venues) :
  try:
    columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
  except:
    columns.append('{}th Most Common Venue'.format(ind+1))

toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]) :
  toronto_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues=num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Cheese Shop,Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Burrito Place,Restaurant
2,Business reply mail Processing CentrE,Light Rail Station,Park,Recording Studio,Farmers Market,Burrito Place,Spa,Auto Workshop,Pizza Place,Smoke Shop,Skate Park
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Boutique,Sculpture Garden,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Ice Cream Shop,Salad Place,Burger Joint,Japanese Restaurant,Fried Chicken Joint


In [88]:
k_cluster = 6
toronto_grouped_clustering = toronto_grouped.drop(['Neighborhood'], axis=1)
kmeans = KMeans(n_clusters=k_cluster, random_state=0).fit(toronto_grouped_clustering)

In [70]:
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data

toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on= 'Neighborhood')

toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Theater,Café,Restaurant,Breakfast Spot,Electronics Store,Spa
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Beer Bar,Hobby Shop,Bank,Bar,Spa,Discount Store,Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Restaurant,Diner,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,Italian Restaurant,American Restaurant,Seafood Restaurant,Cosmetics Shop,Lingerie Store,Creperie
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Final Cluster

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_cluster)
ys = [i + x + (i*x)**2 for i in range(k_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters